# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [12]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [5]:
data = pd.read_csv('/Users/jmg/Documents/IronHack/Labs/Modulo_2/lab-intro-recommender-systems/data/customer_product_sales.csv')

In [6]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [28]:
quant_product = data.groupby(['CustomerID','ProductName']).agg({'Quantity': 'sum'}).reset_index()
quant_product

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [17]:
customer_matrix = pd.pivot_table(data, values='Quantity', 
                                 index = ['ProductName'], columns = ['CustomerID'], 
                                 aggfunc = np.sum, fill_value=0)

customer_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [18]:
squareform(pdist(customer_matrix.T, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [24]:
distances = pd.DataFrame(1/(1 + squareform(pdist(customer_matrix.T, 'euclidean'))), 
                         index=customer_matrix.columns, columns=customer_matrix.columns)

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [29]:
similarities = distances[477].sort_values(ascending=False)[1:]
similarities[:5]

CustomerID
3535    0.083651
3317    0.081210
1072    0.079792
1008    0.079246
639     0.078977
Name: 477, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [36]:
quant_product[quant_product['CustomerID'].isin([3535,3317,1072,1008,639])]

,CustomerID,ProductName,Quantity
452,639,Anchovy Paste - 56 G Tube,1
453,639,Appetizer - Mushroom Tart,1
454,639,Assorted Desserts,1
455,639,Bread - Italian Roll With Herbs,1
456,639,Bread - Raisin Walnut Oval,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [23]:
cityblock = pd.DataFrame(1/(1 + squareform(pdist(customer_matrix.T, 'cityblock'))), 
                         index=customer_matrix.columns, columns=customer_matrix.columns)

cityblock

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.008000,0.009901,0.009174,0.008403,0.008929,0.007752,0.008850,0.008621,0.008696,...,0.000564,0.000596,0.000556,0.000674,0.000542,0.000569,0.000556,0.000554,0.000476,0.000534
200,0.008000,1.000000,0.008130,0.007874,0.006993,0.007576,0.007752,0.007752,0.008065,0.008130,...,0.000563,0.000593,0.000557,0.000675,0.000538,0.000567,0.000558,0.000553,0.000473,0.000531
264,0.009901,0.008130,1.000000,0.008547,0.008850,0.008475,0.007874,0.008403,0.008621,0.008264,...,0.000566,0.000596,0.000556,0.000678,0.000543,0.000576,0.000559,0.000557,0.000479,0.000533
356,0.009174,0.007874,0.008547,1.000000,0.008000,0.008197,0.007752,0.009174,0.008065,0.008130,...,0.000564,0.000595,0.000556,0.000676,0.000540,0.000571,0.000553,0.000556,0.000478,0.000534
412,0.008403,0.006993,0.008850,0.008000,1.000000,0.008065,0.007752,0.008850,0.007937,0.008264,...,0.000561,0.000599,0.000553,0.000674,0.000540,0.000570,0.000552,0.000555,0.000477,0.000536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.000569,0.000567,0.000576,0.000571,0.000570,0.000573,0.000571,0.000573,0.000572,0.000571,...,0.000336,0.000360,0.000322,0.000381,0.000357,1.000000,0.000364,0.000351,0.000298,0.000301
98069,0.000556,0.000558,0.000559,0.000553,0.000552,0.000554,0.000553,0.000556,0.000556,0.000556,...,0.000325,0.000336,0.000322,0.000342,0.000351,0.000364,1.000000,0.000351,0.000308,0.000330
98159,0.000554,0.000553,0.000557,0.000556,0.000555,0.000556,0.000554,0.000557,0.000558,0.000556,...,0.000330,0.000367,0.000333,0.000348,0.000328,0.000351,0.000351,1.000000,0.000317,0.000315
